In [ ]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [ ]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('../log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('infogan')):
    os.mkdir('saved_models_{}'.format('infogan'))
f = open('saved_models_{}/log_collapse1.txt'.format('infogan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import mnist,fashion_mnist,cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, concatenate, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np

class INFOGAN():
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.num_classes = 10
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 72
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []


        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', self.mutual_info_loss]

        # Build and the discriminator and recognition network
        self.discriminator, self.auxilliary = self.build_disk_and_q_net()

        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the recognition network Q
        self.auxilliary.compile(loss=[self.mutual_info_loss],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        gen_input = Input(shape=(self.latent_dim,))
        img = self.generator(gen_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        valid = self.discriminator(img)
        # The recognition network produces the label
        target_label = self.auxilliary(img)

        # The combined model  (stacked generator and discriminator)
        self.combined = Model(gen_input, [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=5, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=5, padding='same'))
        model.add(Activation("tanh"))

        gen_input = Input(shape=(self.latent_dim,))
        img = model(gen_input)

        model.summary()

        return Model(gen_input, img)


    def build_disk_and_q_net(self):

        img = Input(shape=self.img_shape)

        # Shared layers between discriminator and recognition network
        model = Sequential()
        model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=5, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Flatten())

        img_embedding = model(img)

        # Discriminator
        validity = Dense(1, activation='sigmoid')(img_embedding)

        # Recognition
        q_net = Dense(128, activation='relu')(img_embedding)
        label = Dense(self.num_classes, activation='softmax')(q_net)

        # Return discriminator and recognition network
        return Model(img, validity), Model(img, label)


    def mutual_info_loss(self, c, c_given_x):
        """The mutual information metric we aim to minimize"""
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

        return conditional_entropy + entropy

    def sample_generator_input(self, batch_size):
        # Generator inputs
        sampled_noise = np.random.normal(0, 1, (batch_size, 62))
        sampled_labels = np.random.randint(0, self.num_classes, batch_size).reshape(-1, 1)
        sampled_labels = to_categorical(sampled_labels, num_classes=self.num_classes)

        return sampled_noise, sampled_labels

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and categorical labels
                sampled_noise, sampled_labels = self.sample_generator_input(batch_size)
                gen_input = np.concatenate((sampled_noise, sampled_labels), axis=1)

                # Generate a half batch of new images
                gen_imgs = self.generator.predict(gen_input)

                # Train on real and generated data
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)

                # Avg. loss
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator and Q-network
                # ---------------------

                g_loss = self.combined.train_on_batch(gen_input, [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %.2f, acc.: %.2f%%] [Q loss: %.2f] [G loss: %.2f]" % (
                    epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss[1], g_loss[2]))

                if global_step % sample_interval == 0:
                    label_dict, value = self.mode_drop(epoch, global_step)
                    steps.append(global_step)
                    values.append(value)
                #             plt.subplots(1, 1)
        plt.plot(steps, values)
        plt.xlim([0, 40000])
        plt.ylim([0.0, 1.0])
        plt.xlabel('steps')
        plt.ylabel('epochs')
        plt.tick_params(axis='both', which='major', labelsize=13)
        plt.legend(loc='lower right')
        if not os.path.isdir('images_infogan'):
            os.mkdir('images_infogan')
        plt.savefig("images_infogan/mode_drop.png")
        plt.show()

    def mode_drop(self, epoch, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        label_dict, value = calculate_labels(gen_imgs, epoch, global_step)
        f.writelines('\n')
        f.writelines('epoch: ' + str(epoch))
        f.writelines('\n')
        f.writelines('chazhi: %.8f' % (value))
        f.writelines('\n')
        f.writelines('%.8f' % (label_dict[i]) for i in label_dict.keys())

        return label_dict, value

if __name__ == '__main__':
    infogan = INFOGAN()
    infogan.train(epochs=100, batch_size=64, sample_interval=200)
#-*-coding:utf-8-*-